In [70]:
import numpy as np
from numpy import array
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_excel('/Users/luyin/Desktop/project/Q&A.xlsx',header = 0)

In [3]:
l = df['Analyst name'].unique() # 79 unique analyst
dic = {} #create dictionary for questions

for name in l:
    list_ = list(df.loc[df['Analyst name']  == name]['Question'])
    dic[name] = list_

In [100]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS |= {'\xa0','\n', " "}
# STOP_WORDS -= {'who','where','how', 'what', 'when', 'whenever'}
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation
def tokenize(sent):
#     sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
    sent = re.sub('^[0-9]+', '', sent)
#     sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #remove single letter 
    tokens = tokenizer(sent)
    return [(token.text.lower()) for token in tokens if (token.text.lower() not in punctuations and token.is_alpha and token.text.lower() not in {'\xa0', ' '," "})]

#     return [(token.lemma_) for token in tokens if \
#             (token.text.lower() not in punctuations \
#              and token.lemma_ not in STOP_WORDS \
#              and token.pos_ in ('PROPN', 'VERB', 'NOUN'))] # only keep PRONOUN, VERB and NOUN
# # test
# tokens = tokenize(" going to hit them one way or another strong dollar did seem to have a huge impact and y ou, what are you doing? I'm 's what do you and me think or like apples and Apple is looking at buying and bought U.K. startup for $1 billion. '\n' another sentence")
# for token in tokens:
#     print (token)

In [17]:
def tokenize_analyst_q(name):
    all_tokens = []
    for q in dic[name]:
        tokens = tokenize(q)
        all_tokens += tokens
    return all_tokens

In [28]:
tokens = tokenize_analyst_q('Glenn Schorr')

In [29]:
# raw_text

In [30]:
len(tokens)

12024

In [46]:
def build_vocab(input_):
    vocab = sorted(set(input_))
    vocab_to_int = dict((c, i) for i, c in enumerate(vocab))
    int_to_vocab = dict((i, c) for i, c in enumerate(vocab))
    n_total = len(input_)
    n_vocab = len(vocab)
    print ("Total Words: ",n_total)
    print ("Total Vocab: ",n_vocab)
    return n_total, n_vocab

In [60]:
# organize into sequences of tokens
def build_sequence(length, input_):
    sequences = list()
    for i in range(length, len(input_)):
    # select sequence of tokens
        seq = input_[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [61]:
n_total, n_vocab = build_vocab(tokens)
sequences = build_sequence(20, tokens)

Total Words:  12024
Total Vocab:  1615
Total Sequences: 12004


In [65]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# save sequences to file
out_filename = 'Glenn_Schorr_sequences.txt'
save_doc(sequences, out_filename)


In [66]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [80]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model

filepath="LSTM_basline.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=200, epochs=100, callbacks=callbacks_list)

# categorical_crossentropy
# https://cloud.githubusercontent.com/assets/18217467/25556005/1bb51402-2d12-11e7-87ef-a9bad097a858.png
 
# save the model to file
model.save('model.h5')
# save the tokenizer
plk.dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 19, 50)            80800     
_________________________________________________________________
lstm_19 (LSTM)               (None, 19, 100)           60400     
_________________________________________________________________
lstm_20 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 1616)              163216    
Total params: 394,916
Trainable params: 394,916
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
12004/12004 [==============================] - 17s 1ms/step - loss: 6.4232 - acc: 0.0408

Epoch 00001: loss improved fr

12004/12004 [==============================] - 12s 1ms/step - loss: 4.4519 - acc: 0.1426

Epoch 00039: loss improved from 4.47781 to 4.45190, saving model to LSTM_basline.hdf5
Epoch 40/100
12004/12004 [==============================] - 13s 1ms/step - loss: 4.4240 - acc: 0.1478

Epoch 00040: loss improved from 4.45190 to 4.42399, saving model to LSTM_basline.hdf5
Epoch 41/100
12004/12004 [==============================] - 12s 1ms/step - loss: 4.4016 - acc: 0.1465

Epoch 00041: loss improved from 4.42399 to 4.40159, saving model to LSTM_basline.hdf5
Epoch 42/100
12004/12004 [==============================] - 13s 1ms/step - loss: 4.3858 - acc: 0.1497

Epoch 00042: loss improved from 4.40159 to 4.38577, saving model to LSTM_basline.hdf5
Epoch 43/100
12004/12004 [==============================] - 12s 1ms/step - loss: 4.3534 - acc: 0.1519

Epoch 00043: loss improved from 4.38577 to 4.35336, saving model to LSTM_basline.hdf5
Epoch 44/100
12004/12004 [==============================] - 12s 1ms/

12004/12004 [==============================] - 13s 1ms/step - loss: 3.5426 - acc: 0.2213

Epoch 00083: loss improved from 3.56217 to 3.54263, saving model to LSTM_basline.hdf5
Epoch 84/100
12004/12004 [==============================] - 13s 1ms/step - loss: 3.5284 - acc: 0.2266

Epoch 00084: loss improved from 3.54263 to 3.52840, saving model to LSTM_basline.hdf5
Epoch 85/100
12004/12004 [==============================] - 13s 1ms/step - loss: 3.5021 - acc: 0.2283

Epoch 00085: loss improved from 3.52840 to 3.50213, saving model to LSTM_basline.hdf5
Epoch 86/100
12004/12004 [==============================] - 13s 1ms/step - loss: 3.4879 - acc: 0.2298

Epoch 00086: loss improved from 3.50213 to 3.48795, saving model to LSTM_basline.hdf5
Epoch 87/100
12004/12004 [==============================] - 12s 1ms/step - loss: 3.4782 - acc: 0.2304

Epoch 00087: loss improved from 3.48795 to 3.47823, saving model to LSTM_basline.hdf5
Epoch 88/100
12004/12004 [==============================] - 12s 1ms/

In [82]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
    # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

absolutely awesome i do have one tiny follow up i always get a little more than i wanted there thank

that of the past is i just curious if you could give the question but i guess the question but i just curious if you could give the question but i guess the question but i just curious if you could give the question but i guess the question but


In [ ]:
#test 1
# load cleaned text sequences
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [85]:
model.load_weights(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
result_history = model.fit(X, y, batch_size=200, epochs= 500, callbacks=callbacks_list)

Epoch 1/500
12004/12004 [==============================] - 18s 2ms/step - loss: 3.3733 - acc: 0.2468

Epoch 00001: loss did not improve from 3.25588
Epoch 2/500
12004/12004 [==============================] - 12s 1ms/step - loss: 3.2386 - acc: 0.2646

Epoch 00002: loss improved from 3.25588 to 3.23858, saving model to LSTM_basline.hdf5
Epoch 3/500
12004/12004 [==============================] - 12s 1ms/step - loss: 3.2111 - acc: 0.2687

Epoch 00003: loss improved from 3.23858 to 3.21107, saving model to LSTM_basline.hdf5
Epoch 4/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1937 - acc: 0.2726

Epoch 00004: loss improved from 3.21107 to 3.19372, saving model to LSTM_basline.hdf5
Epoch 5/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1613 - acc: 0.2755

Epoch 00005: loss improved from 3.19372 to 3.16126, saving model to LSTM_basline.hdf5
Epoch 6/500
12004/12004 [==============================] - 13s 1ms/step - loss: 3.1517 - acc: 0.279

12004/12004 [==============================] - 13s 1ms/step - loss: 2.6505 - acc: 0.3688

Epoch 00047: loss improved from 2.66119 to 2.65046, saving model to LSTM_basline.hdf5
Epoch 48/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.6528 - acc: 0.3655

Epoch 00048: loss did not improve from 2.65046
Epoch 49/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6265 - acc: 0.3700

Epoch 00049: loss improved from 2.65046 to 2.62651, saving model to LSTM_basline.hdf5
Epoch 50/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6029 - acc: 0.3795

Epoch 00050: loss improved from 2.62651 to 2.60288, saving model to LSTM_basline.hdf5
Epoch 51/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6177 - acc: 0.3751

Epoch 00051: loss did not improve from 2.60288
Epoch 52/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.6046 - acc: 0.3744

Epoch 00052: loss did not improve from 2.60

12004/12004 [==============================] - 15s 1ms/step - loss: 2.2306 - acc: 0.4513

Epoch 00093: loss did not improve from 2.22497
Epoch 94/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.1885 - acc: 0.4607

Epoch 00094: loss improved from 2.22497 to 2.18847, saving model to LSTM_basline.hdf5
Epoch 95/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1967 - acc: 0.4597

Epoch 00095: loss did not improve from 2.18847
Epoch 96/500
12004/12004 [==============================] - 13s 1ms/step - loss: 2.1658 - acc: 0.4667

Epoch 00096: loss improved from 2.18847 to 2.16585, saving model to LSTM_basline.hdf5
Epoch 97/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1459 - acc: 0.4714

Epoch 00097: loss improved from 2.16585 to 2.14593, saving model to LSTM_basline.hdf5
Epoch 98/500
12004/12004 [==============================] - 14s 1ms/step - loss: 2.1837 - acc: 0.4608

Epoch 00098: loss did not improve from 2.14

12004/12004 [==============================] - 15s 1ms/step - loss: 1.8255 - acc: 0.5381

Epoch 00140: loss did not improve from 1.75811
Epoch 141/500
12004/12004 [==============================] - 13s 1ms/step - loss: 1.7580 - acc: 0.5584

Epoch 00141: loss improved from 1.75811 to 1.75800, saving model to LSTM_basline.hdf5
Epoch 142/500
12004/12004 [==============================] - 15s 1ms/step - loss: 1.7543 - acc: 0.5617

Epoch 00142: loss improved from 1.75800 to 1.75430, saving model to LSTM_basline.hdf5
Epoch 143/500
12004/12004 [==============================] - 15s 1ms/step - loss: 1.7451 - acc: 0.5626

Epoch 00143: loss improved from 1.75430 to 1.74511, saving model to LSTM_basline.hdf5
Epoch 144/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.7682 - acc: 0.5536

Epoch 00144: loss did not improve from 1.74511
Epoch 145/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.7500 - acc: 0.5581

Epoch 00145: loss did not improve from

12004/12004 [==============================] - 14s 1ms/step - loss: 1.3776 - acc: 0.6558

Epoch 00188: loss improved from 1.40204 to 1.37761, saving model to LSTM_basline.hdf5
Epoch 189/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.3983 - acc: 0.6526

Epoch 00189: loss did not improve from 1.37761
Epoch 190/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.4129 - acc: 0.6432

Epoch 00190: loss did not improve from 1.37761
Epoch 191/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.3716 - acc: 0.6526

Epoch 00191: loss improved from 1.37761 to 1.37159, saving model to LSTM_basline.hdf5
Epoch 192/500
12004/12004 [==============================] - 92s 8ms/step - loss: 1.3663 - acc: 0.6541

Epoch 00192: loss improved from 1.37159 to 1.36629, saving model to LSTM_basline.hdf5
Epoch 193/500
12004/12004 [==============================] - 13s 1ms/step - loss: 1.3432 - acc: 0.6625

Epoch 00193: loss improved from 1.3662

12004/12004 [==============================] - 14s 1ms/step - loss: 1.0090 - acc: 0.7520

Epoch 00236: loss did not improve from 1.00083
Epoch 237/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.9938 - acc: 0.7585

Epoch 00237: loss improved from 1.00083 to 0.99383, saving model to LSTM_basline.hdf5
Epoch 238/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.0087 - acc: 0.7501

Epoch 00238: loss did not improve from 0.99383
Epoch 239/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.9798 - acc: 0.7617

Epoch 00239: loss improved from 0.99383 to 0.97977, saving model to LSTM_basline.hdf5
Epoch 240/500
12004/12004 [==============================] - 14s 1ms/step - loss: 1.0141 - acc: 0.7485

Epoch 00240: loss did not improve from 0.97977
Epoch 241/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.9675 - acc: 0.7655

Epoch 00241: loss improved from 0.97977 to 0.96751, saving model to LSTM_basl

12004/12004 [==============================] - 14s 1ms/step - loss: 0.7616 - acc: 0.8085

Epoch 00285: loss did not improve from 0.69694
Epoch 286/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7461 - acc: 0.8152

Epoch 00286: loss did not improve from 0.69694
Epoch 287/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7726 - acc: 0.8076

Epoch 00287: loss did not improve from 0.69694
Epoch 288/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7708 - acc: 0.8017

Epoch 00288: loss did not improve from 0.69694
Epoch 289/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7291 - acc: 0.8201

Epoch 00289: loss did not improve from 0.69694
Epoch 290/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7545 - acc: 0.8155

Epoch 00290: loss did not improve from 0.69694
Epoch 291/500
12004/12004 [==============================] - 14s 1ms/step - loss: 0.7521 - acc: 0.8103

Epoc

11800/12004 [============================>.] - ETA: 0s - loss: 0.5160 - acc: 0.8738

KeyboardInterrupt: 

In [93]:
# result_history.history["loss"]

In [87]:
model.save('model.h6')

In [104]:
#test 1
# load cleaned text sequences
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
# model = load_model('model.h6')

model.load_weights(filepath)
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

Initial text:
general to your ability to return that excess capital i heard your comments about redeploying if you not allowed but

Generated text:
it seems like your rwa reduction in housing of commercial is that the last couple of products people you getting a little bit to volatility to client mix of actually seems the worries been trying to irrational this in the ficc are lining color that you saw the reserve as the main activity should icg something the market went average go down and i heard all the opportunities conversation that i curious if retail relates how there any not tradeoffs and revenues maybe far of the same markets and you seems like you do have reserved and jamie to that


In [110]:
#test 2
# load cleaned text sequences
in_filename = 'Glenn_Schorr_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
# model = load_model('model.h6')

model.load_weights(filepath)
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

Initial text:
much of that can fall to the bottom line because there a lot of optimism about what can happen if

Generated text:
stocks have moved well we expecting that to move to the bottom line the big concern that people have is that branded out a lot of people billion the portfolio ticking down a little bit in your comments so i heard all the whole gains in hit in your words
